In [1]:
### step1
# Imports
###
import csv
import json
import os
import pprint
from dictor import dictor
import pandas as pd
from copy import copy
import requests


In [2]:
### step2
# Declare Function to get the right cost period FIXME
##
def getCostPeriod(cost_periods):
    cost_period = {}
    if (len(cost_periods) == 1):
        cost_period = cost_periods[0]
    elif (len(cost_periods) > 1):
        cost_period = cost_periods[0]

    return cost_period

In [18]:
### step3
# fill out the cost values with the coversion to eur
###
def generateCostPaxs16(rowToInsert, cost_period, tripLength, obj):
    providerType = dictor(obj,'provider.provider_type.id')
    expense_type = dictor(obj,'expense_type')
    for i in range(1, 17):
        costPaxBase = cost_period.get('cost_pax_{}'.format(i))
        # used just for one pax
        singleSupercharge = cost_period.get('single_supercharge', 0) if i == 1 else 0
        # used for more than one pax
        singleRoomCostPerPax = cost_period.get('single_room_cost_per_pax', 0) if i == 1 else 0
        # used for more than one pax
        doubleRoomCostPerPax = cost_period.get('double_room_cost_per_pax', 0) if i != 1 else 0

        # get tc_driver_cost_per_pax
        _tcDCostXxPax = cost_period.get('tc_driver_costs_xx_pax', 0)
        _amountTcDXxPax = cost_period.get('amount_of_xx_pax_to_pay_tc_or_driver_costs', 0)
        _tcDCostXxPlusPax = cost_period.get('tc_driver_costs_xx_plus_1_pax', 0)
        try:
            _tcDriverCost = _tcDCostXxPax if i <= _amountTcDXxPax else _tcDCostXxPlusPax
            tcDriverCostPax = _tcDriverCost / i
        except TypeError:
            tcDriverCostPax = 0

        # TC service cost per pax
        _tcSCostPerDay = cost_period.get('tc_cost_per_day', 0)
        _tcSCostPerDeparture = cost_period.get('tc_cost_per_departure', 0)
        try:
            tcServiceCostPax = ((_tcSCostPerDay * (tripLength - 1)) + _tcSCostPerDeparture) / i
        except TypeError:
            tcServiceCostPax = 0

        costPaxUpdated = costPaxBase

        if providerType == 'DMC' or providerType == 'OPERATOR':
            try:
                costPaxUpdated = singleSupercharge + tcDriverCostPax + costPaxBase
            except TypeError:
                costPaxUpdated = 0

        elif providerType == 'ACCOMMODATION' and expense_type == 'HOTEL':
            try:
                costPaxUpdated = singleRoomCostPerPax + doubleRoomCostPerPax + tcDriverCostPax
            except TypeError:
                costPaxUpdated = 0

        elif providerType == 'TC':
            costPaxUpdated = tcServiceCostPax

        # General
        else:
            try:
                costPaxUpdated = tcDriverCostPax + costPaxBase
            except TypeError:
                costPaxUpdated = 0

        rowToInsert['cost_pax_{}'.format(i)] = costPaxUpdated # Add cost in own currency
        exchangeRate = dictor(cost_period, 'currency.exchange_rate_to_eur')

        if costPaxUpdated != None and exchangeRate != None:
            rowToInsert['cost_eur_pax_{}'.format(i)] = costPaxUpdated * exchangeRate # Add cost in EUR currency

    return rowToInsert

In [4]:
### step4
# Declare url and authentication with header secret FIXME protect this
###
headers = {'content-type': 'application/json', 'x-hasura-admin-secret': 'V3ntur4Pr0dH4sur4Tr4v3lC0re2o2o'}
url = 'https://core.venturatravel.org/v1/graphql'

In [5]:
### step5
# Declare the query
###
query = """
{
  service {
    name
    duration_in_h
    expense_type
    payment_type
    get_feedback
    bed_distribution
    service_cost_period {
      valid_from
      valid_until
      contract_url
      single_surcharge
      single_room_cost_per_pax
      double_room_cost_per_pax
      amount_of_xx_pax_to_pay_tc_or_driver_costs
      tc_driver_costs_xx_pax
      tc_driver_costs_xx_plus_1_pax
      tc_cost_per_day
      tc_cost_per_departure
      cost_pax_1
      cost_pax_2
      cost_pax_3
      cost_pax_4
      cost_pax_5
      cost_pax_6
      cost_pax_7
      cost_pax_8
      cost_pax_9
      cost_pax_10
      cost_pax_11
      cost_pax_12
      cost_pax_13
      cost_pax_14
      cost_pax_15
      cost_pax_16
      currency {
        id
        abbreviation
        exchange_rate_to_eur
      }
    }
    provider {
      name
      code
      provider_level {
        name
      }
      provider_type {
        id
        name
      }
    }
    trip_version_days {
      day_number
      accommodation_paid_by_dmc_or_external_op
      trip_version {
        length
        version_name
        trip {
          project {
            name
          }
          code
          name
          trip_type {
            name
          }
          seo {
            locale_id
          }
        }
      }
    }
    trip_version_day_services {
      trip_version_day {
        day_number
        trip_version {
          length
          version_name
          trip {
            project {
              name
            }
            code
            name
            trip_type {
              name
            }
            seo {
              locale_id
            }
          }
        }
      }
    }
    departure_tc_1_services {
      departures {
        trip_version {
          length
          version_name
          trip {
            project {
              name
            }
            code
            name
            trip_type {
              name
            }
            seo {
              locale_id
            }
          }
        }
      }
    }
    trip_versions {
        trip_version {
            length
            version_name
            trip {
              project {
                name
              }
              code
              name
              trip_type {
                name
              }
              seo {
                locale_id
              }
            }
        }
    }
  }
}
"""


In [6]:
### step6
# submit the request and get the results
###
print("Staring request")
r = requests.post(url=url, json={'query': query}, headers=headers)
services = r.json()['data']['service']
print("FInished reuest, {}".format(len(services)))


Staring request
FInished reuest, 1


In [7]:
### step7
# Declare Headers
###
headers = [
        'service_name',
        'provider_name',
        'provider_code',
        'trip_code', # trip dependent
        'provider_type',
        'service_level',
        'duration_in_h',
        'expense_type',
        'payment_type',
        'get_feedback',
        'bed_distribution',
        'contract_link',
        'trip_version_name', # trip dependent
        'market',# trip dependent
        'trip_type',# trip dependent
        'brand',# trip dependent
        'trip_day',# trip dependent
        'currency',
        'cost_pax_1',
        'cost_pax_2',
        'cost_pax_3',
        'cost_pax_4',
        'cost_pax_5',
        'cost_pax_6',
        'cost_pax_7',
        'cost_pax_8',
        'cost_pax_9',
        'cost_pax_10',
        'cost_pax_11',
        'cost_pax_12',
        'cost_pax_13',
        'cost_pax_14',
        'cost_pax_15',
        'cost_pax_16',
        'cost_eur_pax_1',
        'cost_eur_pax_2',
        'cost_eur_pax_3',
        'cost_eur_pax_4',
        'cost_eur_pax_5',
        'cost_eur_pax_6',
        'cost_eur_pax_7',
        'cost_eur_pax_8',
        'cost_eur_pax_9',
        'cost_eur_pax_10',
        'cost_eur_pax_11',
        'cost_eur_pax_12',
        'cost_eur_pax_13',
        'cost_eur_pax_14',
        'cost_eur_pax_15',
        'cost_eur_pax_16',
]


In [17]:
### step8
# Get rows to upload
###
rows = []
for obj in services:
    acc_days = obj.get('trip_version_days', []) # accomodation where the service is used
    tc_days = obj.get('departure_tc_1_services', []) # TC where the service is being used
    day_services = obj.get('trip_version_day_services', []) # other services used (shows in the side)
    trip_version_services = obj.get('trip_versions', []) # other services used at trip_version level (no multiple provider)

    # skip services not being used
    if len(acc_days) == 0 and len(tc_days) == 0 and len(day_services) == 0 and len(trip_version_services) == 0:
        continue

    # common values
    base_obj = {'service_name': dictor(obj, 'name'),
            'provider_name': dictor(obj, 'provider.name'),
            'provider_code': dictor(obj,'provider.code'),
            'provider_type': dictor(obj,'provider.provider_type.name'),
            'service_level': dictor(obj, 'provider.provider_level.name'),
            'duration_in_h': dictor(obj, 'duration_in_h'),
            'expense_type': dictor(obj, 'expense_type'),
            'payment_type': dictor(obj, 'payment_type'),
            'get_feedback': dictor(obj, 'get_feedback'),
            'bed_distribution': dictor(obj, 'bed_distribution')}

    """
    -- trip_version_days --
    Accommodation service related to trip version day
    generally is just one service for every day
    """
    for accm in acc_days:
        # When a tour operator offers a 3 day packages, they may include accommodation.
        # We want to avoid that the accommodation service costs will appear as costs as
        # they are already included in the operator costs. We tell the system that these costs
        # should be ignored, when you check this checkbox
        needsToBeSkiped = dictor(accm, 'accommodation_paid_by_dmc_or_external_op', False)
        if (needsToBeSkiped):
            continue

        rowToInsert = copy(base_obj)

        rowToInsert['trip_code'] = dictor(accm, 'trip_version.trip.code')
        rowToInsert['trip_version_name'] = dictor(accm, 'trip_version.version_name')
        rowToInsert['market'] = dictor(accm, 'trip_version.trip.seo.locale_id')
        rowToInsert['trip_type'] = dictor(accm, 'trip_version.trip.trip_type.name')
        rowToInsert['brand'] = dictor(accm, 'trip_version.trip.project.name')
        rowToInsert['trip_day'] = dictor(accm, 'day_number')
        tripLength = dictor(accm, 'trip_version.length', 0)

        cost_period = getCostPeriod(obj.get('service_cost_period', []))
        rowToInsert['currency'] = dictor(cost_period, 'currency.abbreviation')
        rowToInsert['contract_link'] = dictor(cost_period, 'contract_url')
        rowToInsert = generateCostPaxs16(rowToInsert, cost_period, tripLength, obj)
        rows.append(rowToInsert)

    """
    -- trip_versions --
    Service associated to a trip_version
    used for is just one provider/service from the trip_version admin
    """
    for tvs in trip_version_services:
        rowToInsert = copy(base_obj)

        rowToInsert['trip_code'] = dictor(tvs, 'trip_version.trip.code')
        rowToInsert['trip_version_name'] = dictor(tvs, 'trip_version.version_name')
        rowToInsert['market'] = dictor(tvs, 'trip_version.trip.seo.locale_id')
        rowToInsert['trip_type'] = dictor(tvs, 'trip_version.trip.trip_type.name')
        rowToInsert['brand'] = dictor(tvs, 'trip_version.trip.project.name')
        rowToInsert['trip_day'] = 0
        tripLength = dictor(tvs, 'trip_version.length', 0)

        cost_period = getCostPeriod(obj.get('service_cost_period', []))
        rowToInsert['currency'] = dictor(cost_period, 'currency.abbreviation')
        rowToInsert['contract_link'] = dictor(cost_period, 'contract_url')
        rowToInsert = generateCostPaxs16(rowToInsert, cost_period, tripLength, obj)
        rows.append(rowToInsert)

    """
    -- departure_tc_1_services --
    TC where the service is being used
    used for is just one provider/service from the trip_version admin
    """
    for tcd in tc_days:
        rowToInsert = copy(base_obj)

        rowToInsert['trip_code'] = dictor(tcd, 'departures.0.trip_version.trip.code')
        rowToInsert['trip_version_name'] = dictor(tcd, 'departures.0.trip_version.version_name')
        rowToInsert['market'] = dictor(tcd, 'departures.0.trip_version.trip.seo.locale_id')
        rowToInsert['trip_type'] = dictor(tcd, 'departures.0.trip_version.trip.trip_type.name')
        rowToInsert['brand'] = dictor(tcd, 'departures.0.trip_version.trip.project.name')
        rowToInsert['trip_day'] = 1
        tripLength = dictor(tcd, 'departures.0.trip_version.length', 0)

        cost_period = getCostPeriod(obj.get('service_cost_period', []))
        rowToInsert['currency'] = dictor(cost_period, 'currency.abbreviation')
        rowToInsert['contract_link'] = dictor(cost_period, 'contract_url')
        rowToInsert = generateCostPaxs16(rowToInsert, cost_period, tripLength, obj)
        rows.append(rowToInsert)

    """
    -- trip_version_day_services --
    Other services used (shows in the side in the trip builder)
    """
    for dse in day_services:
        rowToInsert = copy(base_obj)

        rowToInsert['trip_code'] = dictor(dse, 'trip_version_day.trip_version.trip.code')
        rowToInsert['trip_version_name'] = dictor(dse, 'trip_version_day.trip_version.version_name')
        rowToInsert['market'] = dictor(dse, 'trip_version_day.trip_version.trip.seo.locale_id')
        rowToInsert['trip_type'] = dictor(dse, 'trip_version_day.trip_version.trip.trip_type.name')
        rowToInsert['brand'] = dictor(dse, 'trip_version_day.trip_version.trip.project.name')
        rowToInsert['trip_day'] = dictor(dse, 'trip_version_day.day_number')
        tripLength = dictor(dse, 'trip_version_day.trip_version.length', 0)

        cost_period = getCostPeriod(obj.get('service_cost_period', []))
        rowToInsert['currency'] = dictor(cost_period, 'currency.abbreviation')
        rowToInsert['contract_link'] = dictor(cost_period, 'contract_url')
        rowToInsert = generateCostPaxs16(rowToInsert, cost_period, tripLength, obj)
        rows.append(rowToInsert)


{'valid_from': '2021-01-01', 'valid_until': '2021-12-31', 'contract_url': None, 'single_surcharge': None, 'single_room_cost_per_pax': 63, 'double_room_cost_per_pax': 31.5, 'amount_of_xx_pax_to_pay_tc_or_driver_costs': 16, 'tc_driver_costs_xx_pax': 0, 'tc_driver_costs_xx_plus_1_pax': 0, 'tc_cost_per_day': None, 'tc_cost_per_departure': None, 'cost_pax_1': None, 'cost_pax_2': None, 'cost_pax_3': None, 'cost_pax_4': None, 'cost_pax_5': None, 'cost_pax_6': None, 'cost_pax_7': None, 'cost_pax_8': None, 'cost_pax_9': None, 'cost_pax_10': None, 'cost_pax_11': None, 'cost_pax_12': None, 'cost_pax_13': None, 'cost_pax_14': None, 'cost_pax_15': None, 'cost_pax_16': None, 'currency': {'id': 2, 'abbreviation': 'USD', 'exchange_rate_to_eur': 0.82943}}
{'valid_from': '2021-01-01', 'valid_until': '2021-12-31', 'contract_url': None, 'single_surcharge': None, 'single_room_cost_per_pax': 63, 'double_room_cost_per_pax': 31.5, 'amount_of_xx_pax_to_pay_tc_or_driver_costs': 16, 'tc_driver_costs_xx_pax': 0, 

In [9]:
print(len(rows))

17


In [10]:
### step9
# Write file
###
with open('mydata.csv', 'w') as f:
    f_csv = csv.DictWriter(f, headers)
    f_csv.writeheader()
    f_csv.writerows(rows)
df = pd.read_csv('mydata.csv')

In [11]:
df.columns

Index(['service_name', 'provider_name', 'provider_code', 'trip_code',
       'provider_type', 'service_level', 'duration_in_h', 'expense_type',
       'payment_type', 'get_feedback', 'bed_distribution', 'contract_link',
       'trip_version_name', 'market', 'trip_type', 'brand', 'trip_day',
       'currency', 'cost_pax_1', 'cost_pax_2', 'cost_pax_3', 'cost_pax_4',
       'cost_pax_5', 'cost_pax_6', 'cost_pax_7', 'cost_pax_8', 'cost_pax_9',
       'cost_pax_10', 'cost_pax_11', 'cost_pax_12', 'cost_pax_13',
       'cost_pax_14', 'cost_pax_15', 'cost_pax_16', 'cost_eur_pax_1',
       'cost_eur_pax_2', 'cost_eur_pax_3', 'cost_eur_pax_4', 'cost_eur_pax_5',
       'cost_eur_pax_6', 'cost_eur_pax_7', 'cost_eur_pax_8', 'cost_eur_pax_9',
       'cost_eur_pax_10', 'cost_eur_pax_11', 'cost_eur_pax_12',
       'cost_eur_pax_13', 'cost_eur_pax_14', 'cost_eur_pax_15',
       'cost_eur_pax_16'],
      dtype='object')

In [12]:
df.head()

,service_name,provider_name,provider_code,trip_code,provider_type,service_level,duration_in_h,expense_type,payment_type,get_feedback,...,cost_eur_pax_7,cost_eur_pax_8,cost_eur_pax_9,cost_eur_pax_10,cost_eur_pax_11,cost_eur_pax_12,cost_eur_pax_13,cost_eur_pax_14,cost_eur_pax_15,cost_eur_pax_16
0,Comfort room,Hotel Ruinas,NaN,cper,Accommodation,Comfort,NaN,HOTEL,BER,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Comfort room,Hotel Ruinas,NaN,cper,Accommodation,Comfort,NaN,HOTEL,BER,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Comfort room,Hotel Ruinas,NaN,cpbo,Accommodation,Comfort,NaN,HOTEL,BER,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Comfort room,Hotel Ruinas,NaN,cpbo,Accommodation,Comfort,NaN,HOTEL,BER,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Comfort room,Hotel Ruinas,NaN,cper,Accommodation,Comfort,NaN,HOTEL,BER,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
### step10
# Upload data to Bigquery
###
print("to upload {}".format(len(df.index)))

to upload 17


In [14]:
df.to_gbq("ventura_admins.services_v1", "ventura-cluster", if_exists='replace')